# Machine learning: regression

We'll try to predict missing well logs using regression.

The data are from Colorado. We've already loaded the data into a CSV.

In [ ]:
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact

In [ ]:
well_logs = '../data/Colorado_well_data.csv'

In [ ]:
df = pd.read_csv(well_logs, index_col=0)

In [ ]:
df.describe()

Look at the counts: some of the features have some NaNs. It looks like we won't lose too much data by doing a `dropna`...

In [ ]:
df = df.dropna()

In [ ]:
df.describe()

In [ ]:
df.head()

I'd prefer it if the well names were integers not floats.

In [ ]:
df.Well = df.Well.astype(int)

In [ ]:
df.head()

# Visual inspection of the data space

In [ ]:
well = 1

features = ['CAL', 'SP', 'GR', 'RES', 'NPHI', 'RHOB']
target = 'DT'

fig, axs = plt.subplots(ncols=len(features)+1, sharey=True, figsize=(8,8))

for ax, feature in zip(axs, features):
    ax.plot(df.loc[df.Well==well, feature], df.loc[df.Well==well, 'Depth'])
    ax.set_title(feature)
axs[-1].plot(df.loc[df.Well==well, target], df.loc[df.Well==well, 'Depth'], color='red')
axs[-1].set_title(target)
axs[-1].invert_yaxis()

In [ ]:
fig, axs = plt.subplots(ncols=len(features), figsize=(15, 3))

for ax, feature in zip(axs, features):
    ax = sns.distplot(df[feature], ax=ax)
    ax.set_title(feature)
    ax.set_yticks([])

<div class="alert alert-success">
Make a 'log<sub>10</sub> resisitivity' to deal with the usual RES distribution. Call it `LogRes`.
</div>

In [ ]:
df.describe()

In [ ]:
sns.distplot(df.LogRes)

And update the `features` list:

In [ ]:
features.remove('RES')
features.append('LogRes')

<div class="alert alert-success">
Now look at the gamma ray. Are the very high values coming from one well or from lots of places? 

Decide how to fix the gamma ray. For example, you could:

- Remove one or more wells with bad GR.
- Remove only the rows with very high values.
- Clip the GR, e.g. using `pd.Series.clip()`.
</div>

In [ ]:
df.Well[df.GR > 500].unique()

In [ ]:
df.GR[df.Well==18].plot()
plt.axhline(400, c='red')

In [ ]:
sns.distplot(df.GR)

<div class="alert alert-success">
What is causing the long tail in the NPHI data? Is it spikes or a bad log?

Decide how best to fix the NPHI, limiting its range to the interval 0 to 0.5.
</div>

In [ ]:
sns.distplot(df.NPHI)

The distributions should now look something like this:

In [ ]:
fig, axs = plt.subplots(ncols=len(features), figsize=(15, 3))

for ax, feature in zip(axs, features):
    ax = sns.distplot(df[feature], ax=ax)
    ax.set_title(feature)
    ax.set_yticks([])

In [ ]:
df.describe()

# Split the dataset

In [ ]:
df.Well.unique()

How many wells is that?

In [ ]:
len(df.Well.unique())

Let's start by training on the first six wells only.

In [ ]:
n = 8  # We'll come back and change this number.

In [ ]:
df_train = df[df.Well <= n].copy()
df_val = df[(df.Well >= 70) & (df.Well < 85)].copy()   # 12 wells
df_test = df[df.Well >= 85].copy()  # 10 wells

In [ ]:
features = ['GR', 'NPHI', 'RHOB', 'LogRes']
target = 'DT'

## Check the distributions

We'd like to make sure the distributions of the 3 datasets are comparable.

In [ ]:
features

In [ ]:
fig, axs = plt.subplots(ncols=len(features), figsize=(15,3))

for ax, feature in zip(axs, features):
    sns.distplot(df_train[feature], ax=ax)
    sns.distplot(df_val[feature], ax=ax)
    sns.distplot(df_test[feature], ax=ax)
    ax.set_yticklabels([])

## Make `X` and `y`

In [ ]:
X_train = df_train[features].values
y_train = df_train[target].values

X_val = df_val[features].values
y_val = df_val[target].values

X_test = df_test[features].values
y_test = df_test[target].values

## Normalize

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

## Train a model

In [ ]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge

regr = Ridge()

regr.fit(X_train, y_train)

In [ ]:
df_val['DT_pred_LR'] = regr.predict(X_val)

df_val.head()

In [ ]:
def plot_track(df, idx, true, pred):
    fig, ax = plt.subplots(1,1)
    fig.set_size_inches(12,2)
    true = df.loc[df.Well == idx, true]
    pred = df.loc[df.Well == idx, pred]
    depths = df.loc[df.Well == idx, 'Depth']
    ax.plot(depths, true, 'k', lw=1.5)
    ax.plot(depths, pred, 'r', lw=1.5)
    ax.set_xlim(1300, 2400)
    ax.set_ylim(40, 140)
    return

In [ ]:
@interact(idx=(df_val.Well.unique().min(), df_val.Well.unique().max(), 1))
def plot_different_wells(idx):
    plot_track(df_val, idx, 'DT', 'DT_pred_LR')
    return


# Evaluation metrics

Two convenient ways to evaluate regressions are with the $R^2$ score...

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error

r2_score(df_val.DT, df_val.DT_pred_LR)

And the RMS error:

In [ ]:
np.sqrt(mean_squared_error(df_val.DT, df_val.DT_pred_LR))

## Check error distribution

In particular, we want to check that:

1. The errors are normally distributed with a zero mean.
1. The variance of the errors is not correlated with the parameters.

There's some good advice about normality tests in [this article by Jason Brownlee](https://machinelearningmastery.com/a-gentle-introduction-to-normality-tests-in-python/).

First we'll just use visual inspection:

In [ ]:
residuals = df_val['DT_pred_LR'] - df_val['DT']

In [ ]:
sns.distplot(residuals)
plt.axvline(0, color='k', lw=0.5)
plt.axvline(residuals.mean(), color='r')
plt.grid(color='k', alpha=0.15)
plt.show()

#### Normality: QQ plot

A quantile-quantile plot generates an idealized distribution, in this case a Gaussian. The idealized samples are divided into quantiles, then each data point in the sample is paired with a similar member from the idealized distribution. The line `'s'` represents the standard 'normal' distribution.

In [ ]:
from statsmodels.graphics.gofplots import qqplot

qqplot(residuals, line='s')
plt.axvline(0, color='k', lw=0.5)
plt.axhline(0, color='k', lw=0.5)
plt.grid(color='k', alpha=0.15)
plt.show()

#### Normality: Shapiro&ndash;Wilk test

Not convinced about this &mdash; seems like most large samples don't fit. `p` just gets very small.

In [ ]:
from scipy.stats import shapiro

res_shuf = residuals.values
np.random.shuffle(res_shuf)

stat, p = shapiro(res_shuf[:500])
print(f'Statistics = {stat:.3f}, p = {p:.3f}')

alpha = 0.05
if p > alpha:
    print('Sample looks Gaussian (fail to reject H0)')
else:
    print('Sample does not look Gaussian (reject H0)')

#### Homoscedasticity: visual inspection

We want to check that the variance of the errors is not correlated with our parameters.

If they are correlated (if the plots below show points with narrow spread at one end and wide at the other), then there are nonlinearities in the data that are not captured by the model. It could be that outliers are skewing the distribution.

In [ ]:
fig, axs = plt.subplots(ncols=len(features), figsize=(16,4), sharey=True)

for ax, feature in zip(axs, features):
    ax.scatter(df_val[feature], residuals, s=1, alpha=0.1)
    ax.set_xlabel(feature)
    ax.axhline(0, color='k', lw=0.5)
    ax.grid(color='k', alpha=0.15)

Seems like there could be an issue in shales (low NPHI), and in rocks with low HC saturation.

## Coefficients

**If the features have been standardized**, then we can interpret the learned coefficients (or parameters, or weights if you prefer) as feature importance.

In [ ]:
np.set_printoptions(suppress=True)
regr.coef_

In [ ]:
regr.intercept_

<div class="alert alert-success">
Can you make a list of the features ordered by their coefficients?
</div>

In [ ]:
df['InvGardner'] = 108 * df.RHOB**4